# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [598]:
problem = '''Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals: 
You have a target of selling 5500 units. 
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target. 

Employee Hours: 
You have both full-time and part-time employees. 
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours. 
Similarly, like sales, there could be deviations in these working hours. 

Overtime: 
Additionally, your employees might end up working overtime. 
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you. 
P1 to P4 are priority factors for each deviation. 
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities. 
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business.'''

## 2. Generate the mathematical model

In [599]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [600]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Shop Management

**Sets:**

* E = {Full-Time, Part-Time} : Set of employee types

**Parameters:**

*  `sales_target` = 5500  : Target sales units 
*  `full_time_hours_target` = 800 : Target hours for full-time employees
*  `part_time_hours_target` = 320 : Target hours for part-time employees
*  `overtime_target` = 100 : Target overtime hours for full-time employees
*  P1 : Priority factor for sales deviation
*  P2 : Priority factor for full-time employee hours deviation
*  P3 : Priority factor for part-time employee hours deviation
*  P4 : Priority factor for full-time employee overtime deviation

**Decision Variables:**

*  `S` : Actual sales units
*  `FH` : Actual full-time employee hours
*  `PH` : Actual part-time employee hours
*  `FO` :  Actual overtime hours for full-time employees
*  `PO` :  Actual overtime hours for part-time employees

**Objective Function:**

Minimize the total weighted deviation from targets:

```
Minimize:
P1 * |S - sales_target| + 2 * P2 * |FH - full_time_hours_target| + P3 * |PH - part_time_hours_target| + P4 * |FO - overtime_target| + 3 * P4 * PO 
```

**Constraints:**

*  None explicitly stated in the problem. However, you might have implicit constraints like:
    *  Non-negativity: All decision variables should be greater than or equal to zero. 
    *  Maximum working hours: There might be legal or contractual limits on the maximum hours an employee can work.

**Explanation:**

* **Objective Function:** The objective function calculates the weighted sum of deviations from each target. 
    * The absolute value ensures that both positive and negative deviations are penalized.
    * `2 * P2` reflects the double concern for fully utilizing full-time employee hours.
    * `3 * P4 * PO` reflects the higher relative cost of part-time employee overtime.
* **Decision Variables:** These represent the actual values achieved for each goal, which the model will optimize.
* **Constraints:**  You may need to add constraints depending on the specific context of your shop, such as legal limits on working hours or minimum sales requirements. 

This model provides a framework to find the optimal balance between your goals considering their priorities. You can adjust the priority factors (P1-P4) based on your current business needs and priorities. 


## 3. Generate the pyomo code

In [601]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [602]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - You can replace this with your actual data
sales_target = 5500
full_time_hours_target = 800
part_time_hours_target = 320
overtime_target = 100
P1 = 1  # Priority for sales deviation
P2 = 1  # Priority for full-time employee hours deviation
P3 = 1  # Priority for part-time employee hours deviation
P4 = 1  # Priority for full-time employee overtime deviation

# Model
model = pyo.ConcreteModel()

# Sets
model.E = pyo.Set(initialize=['Full-Time', 'Part-Time'])

# Decision Variables
model.S = pyo.Var(domain=pyo.NonNegativeReals) # Actual sales units
model.FH = pyo.Var(domain=pyo.NonNegativeReals) # Actual full-time employee hours
model.PH = pyo.Var(domain=pyo.NonNegativeReals) # Actual part-time employee hours
model.FO = pyo.Var(domain=pyo.NonNegativeReals) # Actual overtime hours for full-time employees
model.PO = pyo.Var(domain=pyo.NonNegativeReals) # Actual overtime hours for part-time employees

# Objective Function
def objective_rule(model):
    return P1 * abs(model.S - sales_target) + 2 * P2 * abs(model.FH - full_time_hours_target) + P3 * abs(model.PH - part_time_hours_target) + P4 * abs(model.FO - overtime_target) + 3 * P4 * model.PO
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints (Add constraints based on your specific context)
# Example: Maximum working hours per week for full-time employees
# model.C1 = pyo.Constraint(expr= model.FH + model.FO <= 40) 

# Solve
solver = pyo.SolverFactory('cbc') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print Results
print(results)
print("Optimal Solution:")
print("Sales (S):", pyo.value(model.S))
print("Full-Time Hours (FH):", pyo.value(model.FH))
print("Part-Time Hours (PH):", pyo.value(model.PH))
print("Full-Time Overtime (FO):", pyo.value(model.FO))
print("Part-Time Overtime (PO):", pyo.value(model.PO)) 
```

**Explanation:**

1. **Import pyomo:** This line imports the Pyomo library for optimization modeling.
2. **Sample Data:**  I've provided sample data for the parameters. You should replace this with your shop's actual data.
3. **Model:** This creates a ConcreteModel object, which is a Pyomo model instance.
4. **Sets:** We define a set `E` to represent the types of employees (Full-Time, Part-Time). This is currently unused but could be useful for extending the model, for example, adding different types of employees.
5. **Decision Variables:** We define the decision variables using `pyo.Var()`.  The `domain=pyo.NonNegativeReals` ensures that these variables can only take non-negative values.
6. **Objective Function:** The `objective_rule` function defines the objective function as described in your problem. It calculates the weighted sum of deviations from the targets.
7. **Constraints:**  This example includes a commented-out constraint to demonstrate how you can add constraints. You should replace this with constraints relevant to your shop's specific rules and limitations.
8. **Solve:**  We create a solver object (here using the free CBC solver) and use it to solve the model.
9. **Print Results:** The code prints the solver's status and the optimal values for each decision variable. 

**Remember:** 

* **Adjust Priority Factors:** Change the values of `P1`, `P2`, `P3`, and `P4` to reflect the priorities for your shop.
* **Add Constraints:** Incorporate constraints relevant to your shop, such as limits on working hours, minimum sales targets, or relationships between variables. 
* **Data Input:**  Replace the sample data with your actual data for more meaningful results. 


## 4. Input problem data and try running the generated code

In [607]:
import pyomo.environ as pyo

# Sample Data - You can replace this with your actual data
sales_target = 5500
full_time_hours_target = 800
part_time_hours_target = 320
overtime_target = 100
P1 = 4  # Priority for sales deviation
P2 = 3  # Priority for full-time employee hours deviation
P3 = 2  # Priority for part-time employee hours deviation
P4 = 1  # Priority for full-time employee overtime deviation

# Model
model = pyo.ConcreteModel()

# Sets
model.E = pyo.Set(initialize=['Full-Time', 'Part-Time'])

# Decision Variables
model.S = pyo.Var(domain=pyo.NonNegativeReals) # Actual sales units
model.FH = pyo.Var(domain=pyo.NonNegativeReals) # Actual full-time employee hours
model.PH = pyo.Var(domain=pyo.NonNegativeReals) # Actual part-time employee hours
model.FO = pyo.Var(domain=pyo.NonNegativeReals) # Actual overtime hours for full-time employees
model.PO = pyo.Var(domain=pyo.NonNegativeReals) # Actual overtime hours for part-time employees

# Objective Function
def objective_rule(model):
    return P1 * abs(model.S - sales_target) + 2 * P2 * abs(model.FH - full_time_hours_target) + P3 * abs(model.PH - part_time_hours_target) + P4 * abs(model.FO - overtime_target) + 3 * P4 * model.PO
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints (Add constraints based on your specific context)
# Example: Maximum working hours per week for full-time employees
# model.C1 = pyo.Constraint(expr= model.FH + model.FO <= 40) 

# Solve
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print Results
print(results)
print("Optimal Solution:")
print("Sales (S):", pyo.value(model.S))
print("Full-Time Hours (FH):", pyo.value(model.FH))
print("Part-Time Hours (PH):", pyo.value(model.PH))
print("Full-Time Overtime (FO):", pyo.value(model.FO))
print("Part-Time Overtime (PO):", pyo.value(model.PO)) 

model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.11.1\x3a Maximum Number of Iterations
      Exceeded.

Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 5
  Sense: unknown
Solver: 
- Status: warning
  Message: Ipopt 3.11.1\x3a Maximum Number of Iterations Exceeded.
  Termination condition: maxIterations
  Id: 400
  Error rc: 0
  Time: 6.8830952644348145
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Solution:
Sales (S): 5499.841889111822
Full-Time Hours (FH): 1113.929810947496
Part-Time Hours (PH): 446.16822729832677
Full-Time Overtime (FO): 127.25233234103736
Part-Time Overtime (PO): 0.033332512312146866


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [608]:
print(response.text)

## Mathematical Optimization Model for Shop Management

**Sets:**

* E = {Full-Time, Part-Time} : Set of employee types

**Parameters:**

*  `sales_target` = 5500  : Target sales units 
*  `full_time_hours_target` = 800 : Target hours for full-time employees
*  `part_time_hours_target` = 320 : Target hours for part-time employees
*  `overtime_target` = 100 : Target overtime hours for full-time employees
*  P1 : Priority factor for sales deviation
*  P2 : Priority factor for full-time employee hours deviation
*  P3 : Priority factor for part-time employee hours deviation
*  P4 : Priority factor for full-time employee overtime deviation

**Decision Variables:**

*  `S` : Actual sales units
*  `FH` : Actual full-time employee hours
*  `PH` : Actual part-time employee hours
*  `FO` :  Actual overtime hours for full-time employees
*  `PO` :  Actual overtime hours for part-time employees

**Objective Function:**

Minimize the total weighted deviation from targets:

```
Minimize:
P1 * |S - sa

In [609]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - You can replace this with your actual data
sales_target = 5500
full_time_hours_target = 800
part_time_hours_target = 320
overtime_target = 100
P1 = 1  # Priority for sales deviation
P2 = 1  # Priority for full-time employee hours deviation
P3 = 1  # Priority for part-time employee hours deviation
P4 = 1  # Priority for full-time employee overtime deviation

# Model
model = pyo.ConcreteModel()

# Sets
model.E = pyo.Set(initialize=['Full-Time', 'Part-Time'])

# Decision Variables
model.S = pyo.Var(domain=pyo.NonNegativeReals) # Actual sales units
model.FH = pyo.Var(domain=pyo.NonNegativeReals) # Actual full-time employee hours
model.PH = pyo.Var(domain=pyo.NonNegativeReals) # Actual part-time employee hours
model.FO = pyo.Var(domain=pyo.NonNegativeReals) # Actual overtime hours for full-time employees
model.PO = pyo.Var(domain=pyo.NonNegativeReals) # Actual overtime hours for part-time employees

# Objective Function
def objective_ru